# Notebook for tf-idf information retrieval

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from src import init_data
from arqmath_code.topic_file_reader import Topic
from arqmath_code.Entities.Post import Post
import re

ImportError: cannot import name 'init_data' from 'src' (unknown location)

In [ ]:
topic_reader, data_reader = init_data(task=1)

reading users
reading comments
reading votes


In [8]:
test_topic: Topic = topic_reader.get_topic('A.301')

## Remove html tags
As we are not interested in their word counts in the actual tf-idf vectors

In [12]:
test_topic.title = re.sub(r"</?(p|span)[^>]*>", "", test_topic.title)
test_topic.question = re.sub(r"</?(p|span)[^>]*>", "", test_topic.question)
test_topic.question

"Suppose $A$ is a $m\\times n$ matrix. Then Prove that, $\\begin{equation*} \\|A\\|_2\\leq \\sqrt{\\|A\\|_1 \\|A\\|_{\\infty}} \\end{equation*}$ I have proved the following relations: $\\begin{align*} \\frac{1}{\\sqrt{n}}\\|A\\|_{\\infty}\\leq \\|A\\|_2\\leq\\sqrt{m}\\|A\\|_{\\infty}\\\\ \\frac{1}{\\sqrt{m}}\\|A\\|_{1}\\leq \\|A\\|_2\\leq\\sqrt{n}\\|A\\|_{1} \\end{align*}$ Also I feel that somehow Holder's inequality for the special case when $p=1$ and $q=\\infty$ might be useful.But I couldn't prove that. Edit: I would like to have a prove that do not use the information that $\\|A\\|_2=\\sqrt{\\rho(A^TA)}$ Usage of inequalities like Cauchy Schwartz or Holder is fine. "

In [15]:
def remove_html_tags_from_post(post: Post):
    post.body = re.sub(r"</?(p|span)[^>]*>", "", post.body)

## Retrieve Subset of Document Collection by tags
Equivalent to a binary retrieval of documents by only considering tags

In [17]:
potential_questions = [question for tag in test_topic.lst_tags for question in data_reader.get_question_of_tag(tag=tag)]
potential_questions

['linear-algebra',
 'linear-algebra',
 'linear-algebra',
 'linear-algebra',
 'linear-algebra',
 'linear-algebra',
 'linear-algebra',
 'linear-algebra',
 'linear-algebra',
 'linear-algebra',
 'linear-algebra',
 'linear-algebra',
 'linear-algebra',
 'linear-algebra',
 'linear-algebra',
 'linear-algebra',
 'linear-algebra',
 'linear-algebra',
 'linear-algebra',
 'linear-algebra',
 'linear-algebra',
 'linear-algebra',
 'linear-algebra',
 'linear-algebra',
 'linear-algebra',
 'linear-algebra',
 'linear-algebra',
 'linear-algebra',
 'linear-algebra',
 'linear-algebra',
 'linear-algebra',
 'linear-algebra',
 'linear-algebra',
 'linear-algebra',
 'linear-algebra',
 'linear-algebra',
 'linear-algebra',
 'linear-algebra',
 'linear-algebra',
 'linear-algebra',
 'linear-algebra',
 'linear-algebra',
 'linear-algebra',
 'linear-algebra',
 'linear-algebra',
 'linear-algebra',
 'linear-algebra',
 'linear-algebra',
 'linear-algebra',
 'linear-algebra',
 'linear-algebra',
 'linear-algebra',
 'linear-alg